# 搭建向量数据库

## 加载环境变量

In [ ]:
import os

from dotenv import find_dotenv, load_dotenv

# 读取本地/项目的环境变量。
# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
env = load_dotenv(find_dotenv())
os.environ["LANGCHAIN_API_KEY"]

## 读取数据

In [ ]:
file_paths = []
folder_path = './knowledge_data'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
file_paths

## 安装PDF读取工具

In [ ]:
!pip install langchain-community pypdf

## 读取PDF文件

In [ ]:
import re
from langchain.document_loaders import PyPDFLoader

loaders = []

for file_path in file_paths[1:4]:
    loader = PyPDFLoader(file_path)
    loaders.append(loader)


# 清洗数据
def clear_content(content):
    pattern = re.compile(r'[^\u4e00-\u9fff](\n)[^\u4e00-\u9fff]', re.DOTALL)
    content = re.sub(pattern, lambda match: match.group(0).replace('\n', ''), content)
    return content


texts = []
for loader in loaders:
    docs = loader.load()
    for doc in docs:
        doc.page_content = clear_content(doc.page_content)
        texts.append(doc)
texts

## 文档切割

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 切分文档
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50)

split_docs = text_splitter.split_documents(texts)

In [ ]:
split_docs[0]

## 创建向量数据库

In [ ]:
!pip install chromadb

In [ ]:
from langchain.vectorstores import Chroma

from langchain_openai import OpenAIEmbeddings

base_url = os.environ["BASE_URL"]
api_key = os.environ["API_KEY"]
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-large", base_url=base_url, api_key=api_key)

persist_directory = './chroma_db'
db = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings_model,
    persist_directory=persist_directory
)

## 向量检索

In [ ]:
question = "什么是数据结构？"

In [ ]:
# 当你需要数据库返回严谨的按余弦相似度排序的结果时可以使用similarity_search函数。
similar_docs = db.similarity_search(question, k=3)
print("相似度最高的文档数量:", len(similar_docs))

In [ ]:
for i, doc in enumerate(similar_docs):
    print(f"第{i + 1}个文档：\n\n{doc.page_content}\n\n")

## 最大边际检索

如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。

最大边际相关性 (MMR, Maximum marginal relevance) 可以帮助我们在保持相关性的同时，增加内容的丰富度。

核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。

In [ ]:
mmr_docs = db.max_marginal_relevance_search(question, k=3)

In [ ]:
for i, doc in enumerate(mmr_docs):
    print(f"第{i + 1}个文档：\n\n{doc.page_content}\n\n")

## 封装为chain

In [36]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return db.similarity_search(query, k=1)


retriever.batch(
    [
        "冒泡排序是什么",
        "递归是什么",
    ],
)

[[Document(metadata={'page': 1, 'source': './knowledge_data\\03_排序.pdf'}, page_content='1.2 \n冒泡排序  \n冒泡排序（ Bubble Sort），是一种计算机科学领域的较简单的排序算法。 \n需求：  \n \n排序前： {4,5,6,3,2,1} \n  \n  \npu\nbl\nic\n v\noi\nd \nse\ntA\nge\n(i\nnt\n a\nge\n) \n{ \n  \n  \n  \n t\nhi\ns.\nag\ne \n= \nag\ne;\n  \n  \n} \n  \n  \n@O\nve\nrr\nid\ne \n  \n p\nub\nli\nc \nSt\nri\nng\n t\noS\ntr\nin\ng(\n) \n{ \n  \n  \n  \n r\net\nur\nn \n"S\ntu\nde\nnt\n{"\n +\n  \n  \n  \n  \n  \n  \n  \n  \n"u\nse\nrn\nam\ne=\n\'"\n +\n u\nse\nrn\nam\ne \n+ \n\'\\\n\'\'\n +\n  \n  \n  \n  \n  \n  \n  \n  \n",\n a\nge\n="\n +\n a\nge\n +\n  \n  \n  \n  \n  \n  \n  \n  \n\'}\n\';\n  \n  \n} \n  \n /\n/\n定 \n义 \n比 \n较 \n规 \n则  \n  \n @\nOv\ner\nri\nde\n  \n  \npu\nbl\nic\n i')],
 [Document(metadata={'page': 13, 'source': './knowledge_data\\03_排序.pdf'}, page_content='在递归中，不能无限制的调用自己，必须要有边界条件，能够让递归结束，因为每一次递归调用都会在栈内存开辟 \n新的空间，重新执行方法，如果递归的层级太深，很容易造成栈内存溢出。 \n需求：  \n \n请定义一个方法，使用递归完成求 N\n的阶乘；  \n代码实现： \n分 \n析 ：  \n1!\n: \n  \n  \n12\n!